# Objective 1: Create a schema validator

In order to build quickly schema for a Json file, including Json objects with large number of items,and also having better uderstanding, about data types of json object items , we can read Json file and transfer it into the Pandas Data Frame. Based on detected data type in Pandas DF we are able to build our schema very fast with Pandera library of python.In cases that Json file size is large we can read only few objects e.g first 10 objects and transfer them into Pandas Data Frame. 

In [193]:
import json
import pandas as pd
with open('input.json', 'r') as file:
    file = "[" + ",".join(file) + "]"
Json_df = pd.read_json(file)
Json_df=Json_df.convert_dtypes()
Json_df.head()

,id,received_at,anonymous_id,context_app_version,context_device_ad_tracking_enabled,context_device_manufacturer,context_device_model,context_device_type,context_locale,context_network_wifi,...,event_text,original_timestamp,sent_at,timestamp,user_id,context_network_carrier,context_device_token,context_traits_taxfix_language,context_library_name,context_library_version
0,FB16866D-AE4D-416F-8848-122B07DA42F5,2018-01-30 18:13:52.221,0A52CDC6-DDDC-4F7D-AA24-4447F6AF2689,1.2.3,1,Apple,"iPhone8,4",android,de-DE,True,...,submissionSuccess,2018-01-30T19:13:43.383+0100,2018-01-30 18:13:51,2018-01-30 18:13:43.627,18946,o2-de,<NA>,en-DE,<NA>,<NA>
1,AED96FC7-19F1-46AB-B79F-D412117119BD,2018-02-03 18:28:12.378,8E0302A3-2184-4592-851D-B93C32E410AB,<NA>,<NA>,Apple,"iPhone8,4",ios,de-DE,True,...,registrationInitiated,2018-02-03T19:28:06.291+0100,2018-02-03 18:28:12,2018-02-03 18:28:06.561,<NA>,o2-de,<NA>,de-DE,analytics-ios,3.6.7


In [194]:
Json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   id                                  2 non-null      string        
 1   received_at                         2 non-null      datetime64[ns]
 2   anonymous_id                        2 non-null      string        
 3   context_app_version                 1 non-null      string        
 4   context_device_ad_tracking_enabled  1 non-null      Int64         
 5   context_device_manufacturer         2 non-null      string        
 6   context_device_model                2 non-null      string        
 7   context_device_type                 2 non-null      string        
 8   context_locale                      2 non-null      string        
 9   context_network_wifi                2 non-null      boolean       
 10  context_os_name               

In [195]:
Json_df['context_network_wifi'] = Json_df['context_network_wifi'].astype(bool)

In [210]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check, Index
schema_JsonFile= DataFrameSchema(
    {
        "id": Column(str),
        "received_at": Column('datetime64[ns]'),
        "anonymous_id": Column(str),
        "context_app_version": Column(str),
        "context_device_ad_tracking_enabled": Column(int),
        "context_device_manufacturer": Column(str),
        "context_device_model": Column(str),
        'context_device_type': Column(str),
        'context_locale': Column(str),
        'context_network_wifi': Column(bool),
        'context_os_name': Column(str),
        'context_timezone': Column(str),
        'event': Column(str),
        'event_text': Column(str),
        'original_timestamp': Column(str) ,
        'sent_at': Column('datetime64[ns]'), 
         'timestamp': Column('datetime64[ns]') , 
         'user_id': Column(int) ,
        'context_network_carrier': Column(str) ,
         'context_device_token': Column(int) ,
        'context_traits_taxfix_language': Column(str) , 
         'context_library_name': Column(str) ,
         'context_library_version': Column(str),                             
    },
    index=Index(int),
    strict=True,
    coerce=True,
)


# Reading one by one objects from json file and loging validation status for each object 

In [212]:

def validateJson(jsonData):
    try:
        df = pd.DataFrame(jsonData)
        return schema_JsonFile.validate(df)
    except:
        return "->Something went wrong when validating the bject"

print("Starting schema validation process for JsonFile")
print("------------------------------------------------------------------------------")
try:
    with open('input.json','r') as jsonFile:
        for Obj in jsonFile:
            try:
                JasonObject_Dic = json.loads(Obj)
                isValid= validateJSON(Obj)
                print("->JSON object with id:", JasonObject_Dic['id'], " is Valid: ", isValid)
                
            except:
                 print("->Something went wrong when reading the next object from jsonfile")
except:
    print("->Something went wrong when opening the jsonfile")                   
finally:
    print("Process finished successfully")

Starting schema validation process for JsonFile
------------------------------------------------------------------------------
->JSON object with id: FB16866D-AE4D-416F-8848-122B07DA42F5  is Valid:  True
->JSON object with id: AED96FC7-19F1-46AB-B79F-D412117119BD  is Valid:  True
Process finished successfully


# Objective 2: Generate a report with count of each “event” in the file

In [2]:
import json
import pandas as pd
with open('input.json', 'r') as file:
    file = "[" + ",".join(file) + "]"
Json_df = pd.read_json(file)
Json_df=Json_df.convert_dtypes()
Json_df.head()

,id,received_at,anonymous_id,context_app_version,context_device_ad_tracking_enabled,context_device_manufacturer,context_device_model,context_device_type,context_locale,context_network_wifi,...,event_text,original_timestamp,sent_at,timestamp,user_id,context_network_carrier,context_device_token,context_traits_taxfix_language,context_library_name,context_library_version
0,FB16866D-AE4D-416F-8848-122B07DA42F5,2018-01-30 18:13:52.221,0A52CDC6-DDDC-4F7D-AA24-4447F6AF2689,1.2.3,1,Apple,"iPhone8,4",android,de-DE,True,...,submissionSuccess,2018-01-30T19:13:43.383+0100,2018-01-30 18:13:51,2018-01-30 18:13:43.627,18946,o2-de,<NA>,en-DE,<NA>,<NA>
1,AED96FC7-19F1-46AB-B79F-D412117119BD,2018-02-03 18:28:12.378,8E0302A3-2184-4592-851D-B93C32E410AB,<NA>,<NA>,Apple,"iPhone8,4",ios,de-DE,True,...,registrationInitiated,2018-02-03T19:28:06.291+0100,2018-02-03 18:28:12,2018-02-03 18:28:06.561,<NA>,o2-de,<NA>,de-DE,analytics-ios,3.6.7


In [3]:
Json_df['received_at']=Json_df['received_at'].dt.date

In [4]:
Report_Count_Events=pd.DataFrame({'count' : Json_df.groupby(['received_at', 'event']).size()}).reset_index()
Report_Count_Events

,received_at,event,count
0,2018-01-30,submission_success,1
1,2018-02-03,registration_initiated,1


# Assume that the input.json file cannot be loaded into memory all at once 

we read line by line objects from input.json in iteration, every iteration step includes iteration over maximum 10 objects and algorithm continue untill end of the JasonFile ) in this case with do not need to lead, entire of file at once.

In [28]:
import json
import pandas as pd
batch=10
index=0
objcstList=[]
with open('input.json','r') as jsonFile:
    try:
        for Obj in jsonFile:
            if index<10:
                try:
                    objcstList.append(json.loads(Obj)) 
                except:
                     print("->Something went wrong when reading the next object from jsonfile")
                index+=1
            else:
                index=0
    except:
        print("There is no more object in JasonFlie")
DataFrame=pd.DataFrame(objcstList)
DataFrame=DataFrame.convert_dtypes()
DataFrame['received_at'] = DataFrame['received_at'].astype('datetime64[ns]')
DataFrame['received_at']=DataFrame['received_at'].dt.date
Report_Count_Events=pd.DataFrame({'count' : Json_df.groupby(['received_at', 'event']).size()}).reset_index()
Report_Count_Events

,received_at,event,count
0,2018-01-30,submission_success,1
1,2018-02-03,registration_initiated,1
